#Paquetes necesarios

In [1]:
import cv2  
import math 

from ultralytics import YOLO
import easyocr
import pytesseract
import matplotlib.pyplot as plt
import imutils
import numpy as np



In [2]:
# Función para leer matrículas usando easyOCR y procesamiento de imágenes
def readLicensePlate(image):

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    _, thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    reader = easyocr.Reader(['en'])

    results = reader.readtext(thresh)

    print(results)

    cv2.imshow('OCR Result', thresh)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    return results

In [3]:
# Función para determinar la ubicación de la matrícula
def searchLicensePlate(image):
    sizes = image.shape
    imgcpy = image
    result = 0
    print(sizes[0]/sizes[1])
    
    # Ancho
    if sizes[0]/sizes[1] < 0.6:
        results = []
        # Derecha
        imgcpy = resizeImage(image, 0.4, 0, 0.2, 0.2)
        results1 = readLicensePlate(imgcpy)
        if results1:
            results.append(results1)
        imgcpy = image
        # Centro
        imgcpy = resizeImage(image, 0.2, 0.2, 0.2, 0.2)
        results2 = readLicensePlate(imgcpy)
        if results2:
            results.append(results2)
        imgcpy = image
        # Izquierda
        imgcpy = resizeImage(image, 0, 0.4, 0.2, 0.2)
        results3 = readLicensePlate(imgcpy)
        if results3:
            results.append(results3)

        resultsMax = 0
        for r in results:
            if r[0][2] > resultsMax:
                resultsMax = r[0][2]
        for r in results:
            if r[0][2] == resultsMax:
                result = r
    # Alto
    elif sizes[0]/sizes[1] > 1.4:
        image = resizeImage(image, 0.2, 0.2, 0.6, 0.1)
        result = readLicensePlate(image)
    # Cuadrado
    else:
        image = resizeImage(image, 0.2, 0.2, 0.4, 0.1)
        result = readLicensePlate(image)
    print(result)

# Función para redimensionar una imagen porcentualmente por cada uno de sus 4 lados
def resizeImage(image, leftToRight, rightToLeft, topToBottom, bottomToTop):
    sizes = image.shape
    subImg = image[round(sizes[0]*topToBottom):sizes[0]-round(sizes[0]*bottomToTop),round(sizes[1]*leftToRight):sizes[1]-round(sizes[1]*rightToLeft)]

    return subImg

# Función para usar un modelo de YOLO
def useModel(pathModel, img, license):
    model = YOLO(pathModel)
    resultsModel = model(img, stream=True)
    boxesFiltered = []
    for r in resultsModel:
        boxes = r.boxes
        for box in boxes:
            cls = int(box.cls[0])
            if not license:
                if cls != 2:
                    continue
            boxesFiltered.append(box)
    return boxesFiltered

# Función para mostrar los resultados obtenidos de un modelo YOLO
def printResultsInImage(boxes, img):
    i = 0
    for box in boxes:
        x1, y1, x2, y2 = box.xyxy[0]
        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

        cls = int(box.cls[0])

        if cls == 2:
            # print("Class name --> car")
            cv2.putText(img, 'car', [x1, y1], cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)
        else:
            # print("Class name --> license")
            cv2.putText(img, 'license', [x1, y1], cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)

        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 0, 0), 3)
            
        subImg = img[y1:y2,x1:x2]
        searchLicensePlate(subImg)

        i += 1
        cv2.imwrite('images/result' + str(i) + '.jpg', subImg)

    cv2.imwrite('images/result.jpg', img)
    # cv2.imshow('', img)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()

img = cv2.imread('images/images/Cars0.png')
boxes = useModel('yolov8n.pt', img, False)
printResultsInImage(boxes, img)


0: 352x640 1 car, 110.0ms
Speed: 4.0ms preprocess, 110.0ms inference, 2.0ms postprocess per image at shape (1, 3, 352, 640)
Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


0.5284552845528455
[([[45, 71], [226, 71], [226, 123], [45, 123]], 'Klg2CA2555', 0.2791294109466633)]


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


[([[145, 71], [296, 71], [296, 122], [145, 122]], 'Klg1C425', 0.1803451281458457)]


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


[]
[([[45, 71], [226, 71], [226, 123], [45, 123]], 'Klg2CA2555', 0.2791294109466633)]


In [4]:
# Función para mezclar modelos de YOLO, de detección de vehículos y de matrículas y así obtener resultados más precisos
def mixModels(img):
    boxesCars = useModel('yolov8n.pt', img, False)
    results = []
    imgcpy = img

    for box in boxesCars:
        x1, y1, x2, y2 = box.xyxy[0]
        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
        x1_1, y1_1, x2_1, y2_1 = int(x1), int(y1), int(x2), int(y2)

        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 0, 0), 3)

        subImg = img[y1:y2,x1:x2]
        # cv2.imshow('car', subImg)
        # cv2.waitKey(0)
        # cv2.destroyAllWindows()

        boxesLicenses = useModel('yolov8n-licenseTrained.pt', subImg, True)
        for boxL in boxesLicenses:
            x1, y1, x2, y2 = boxL.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
        
            licenseImg = subImg[y1:y2,x1:x2]
            results = readLicensePlate(licenseImg)
            if results:
                print(results[0][1])
                cv2.rectangle(subImg, (x1, y1), (x2, y2), (0, 255, 0), 3)
                cv2.putText(subImg, results[0][1] + '(' + str(round(results[0][2]*100, 2)) + '%)', (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

            imgcpy[y1_1:y2_1, x1_1:x2_1] = subImg

        results.extend(boxesLicenses)

    
    # cv2.imshow('license', imgcpy)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    
    return results
        
        

img = cv2.imread('images/images/Cars1.png')
boxes = mixModels(img)


0: 416x640 2 cars, 1 truck, 112.1ms
Speed: 2.0ms preprocess, 112.1ms inference, 3.0ms postprocess per image at shape (1, 3, 416, 640)

0: 640x320 (no detections), 85.7ms
Speed: 1.0ms preprocess, 85.7ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 320)

0: 416x640 1 licence, 75.4ms
Speed: 2.0ms preprocess, 75.4ms inference, 1.0ms postprocess per image at shape (1, 3, 416, 640)
Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


[([[12, 0], [123, 0], [123, 27], [12, 27]], 'PG  MNN112', 0.11636282628909486)]
PG  MNN112


In [5]:
# Función para pobar el modelo mixto en un vídeo
def videoTest(pathVideo, pathOutputVideo):
    cap = cv2.VideoCapture(pathVideo)

    if not cap.isOpened():
        print("Error al abrir el vídeo")    
        exit()

    width = int(cap.get(3))
    height = int(cap.get(4))
    fps = cap.get(5)

    fourcc = cv2.VideoWriter_fourcc(*'H264')
    out = cv2.VideoWriter(pathOutputVideo, fourcc, fps, (width, height))
    
    nFrames = 0
    while True:
        ret, frame = cap.read()

        if not ret:
            break
        
        nFrames += 1
        if nFrames % 100 == 0:
            mixModels(frame)
            print(nFrames)
            cv2.imshow('Video', frame)
            key = cv2.waitKey(0) & 0xFF
            
        # mixModels(frame)
        # out.write(frame)
        
        cv2.imshow('Video', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    # out.release()
    cv2.destroyAllWindows()
    print(nFrames)


videoTest('videos/Traffic IP Camera.mp4', 'videos/test1.mp4')

98
